# Experiment #1

Let's start with a simple scenario... the wind velocity rotates in a circle and the agent uses uniform probability functions.

* Winds: *Oscillatory*
    * Description of the wind behavior: the wind velocity vector traces out a circle in the XY plane blowing at a constant speed of 10 m/s and changing direction at a rate of ~57 degrees (1 radian) per second.
    * base_velocity = (0, 0, 0) m/s
    * amplitude = (10, 10, 0) m/s
    * frequency = (1, 1, 0) rad/s
    * phase_offset = (0, pi/2, 0) rad
* Aiming geometry: *Spherical*
* Skilled probability distributions: *Uniform* (for timing, speed, & aiming)
    * timing: [0, 2] s
    * speed: [40, 44] m/s
    * aiming:
        * Theta (X1): [-10, 10] deg --> [-0.1745329, 0.1745329] rad
        * Phi (X2): [42, 48] deg --> [0.7330383, 0.837758] rad
* Simulation sampling:
    * num_runs = 10,000
    * num_workers = 1

## Step 1: Generate Winds Spacetime

## Step 2: Run Simulation Experiment

## Step 3: Analyze Results